In [ ]:
import json
import requests
from typing import List

import numpy as np

In [ ]:
SERVER_URL = "[paste server url here]"
TEAM_TOKEN = "[paste your team token here]"

### 1. Model stealing

In [ ]:
def model_stealing(path_to_png_file: str) -> np.ndarray:
    """Send querry to API

    Args:
        path_to_png_file (str): Path to image to send to API (as STRING)

    Raises:
        Exception: In case of request failure

    Returns:
        np.ndarray: Image representation
    """
    
    ENDPOINT = "/modelstealing"
    URL = SERVER_URL + ENDPOINT

    with open(path_to_png_file, "rb") as img_file:
        response = requests.get(
            URL, files={"file": img_file}, headers={"token": TEAM_TOKEN}
        )

        if response.status_code == 200:
            # return response.content["representation"] # obsolete
            return json.loads(response.content.decode())["representation"]
        else:
            raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")


def model_stealing_reset() -> None:
    """Resets API

    Raises:
        Exception: In case of query failure
    """
    
    ENDPOINT = "/modelstealing/reset"
    URL = SERVER_URL + ENDPOINT

    response = requests.post(
        URL, headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        print("Endpoint rested successfully")
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")


def model_stealing_submission(path_to_onnx_file: str) -> float:
    """Submits solution

    Args:
        path_to_onnx_file (str): Path to saved model (as STRING, in ONNX format)

    Raises:
        Exception: In case of query failure

    Returns:
        float: Solution score
    """

    ENDPOINT = "/modelstealing/submit"
    URL = SERVER_URL + ENDPOINT

    with open(path_to_onnx_file, "rb") as onnx_file:
        response = requests.post(
            URL, files={"file": onnx_file}, headers={"token": TEAM_TOKEN}
        )

        if response.status_code == 200:
            return response.content["score"]
            # return json.loads(response.content.decode())["score"]
        else:
            raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

### 2. Sybil attack

In [ ]:
def sybil_attack(ids: List[int], home_or_defense: str, binary_or_affine: str) -> np.ndarray:
    """_summary_

    Args:
        ids (List[int]): ID(-s) of images to encode
        home_or_defense (str): Endpoint type ["home", "defense"]
        binary_or_affine (str): Encoding type ["binary", "affine"]

    Raises:
        Exception: In case of query failure

    Returns:
        np.ndarray: Image(-s) representation
    """
    
    if home_or_defense not in ["home", "defense"] or binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")
    
    ENDPOINT = f"/sybil/{binary_or_affine}/{home_or_defense}"
    URL = SERVER_URL + ENDPOINT
    
    ids = ids = ",".join(map(str, ids))

    response = requests.get(
        URL, params={"ids": ids}, headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        return response.content["representations"]
        # return json.loads(response.content.decode())["representations"]
    
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")


def sybil_attack_reset(binary_or_affine: str) -> None:
    """Resets API
    
    Args:
        binary_or_affine (str): Encoding type ["binary", "affine"]

    Raises:
        Exception: In case of query failure
    """
    
    if binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")
    
    ENDPOINT = f"/sybil/{binary_or_affine}/reset"
    URL = SERVER_URL + ENDPOINT

    response = requests.post(
        URL, headers={"token": TEAM_TOKEN}
    )

    if response.status_code == 200:
        print("Endpoint rested successfully")
    else:
        raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")


def sybil_attack_submission(path_to_npz_file: str, binary_or_affine: str) -> float:
    """Submits solution 

    Args:
        path_to_npz_file (str): Path to encoded representations (as STRING)
        binary_or_affine (str): Encoding type ["binary", "affine"]

    Raises:
        Exception: In case of query failure

    Returns:
        float: Solution score
    """
    
    if binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")

    ENDPOINT = f"/sybil/{binary_or_affine}/submit"
    URL = SERVER_URL + ENDPOINT

    with open(path_to_npz_file, "rb") as npz_file:
        response = requests.post(
            URL, files={"file": npz_file}, headers={"token": TEAM_TOKEN}
        )

        if response.status_code == 200:
            return response.content["score"]
            # return json.loads(response.content.decode())["score"]
        else:
            raise Exception(f"Request failed. Status code: {response.status_code}, content: {response.content}")

### 3. Defense transformation

In [ ]:
def defense_submit(path_to_npz_file: str) -> float:
    """Submit solution

    Args:
        path_to_npz_file (str): Path to encoded representations

    Raises:
        Exception: In case of query failure

    Returns:
        float: Solution score
    """
    
    ENDPOINT = "/defense/submit"
    URL = SERVER_URL + ENDPOINT
    
    with open(path_to_npz_file, "rb") as f:
        response = requests.post(
            URL, files={"file": f}, headers={"token": TEAM_TOKEN}
        )
    
    if response.status_code == 200:
        print("Request OK")
        print(response.json())
        # return json.loads(response.content.decode())["score"]
    else:
        raise Exception(
            f"Defense submit failed. Code: {response.status_code}, content: {response.json()}"
        )